In [1]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
%matplotlib inline

In [2]:
nba = pd.read_csv('AllStarNBA')
nba_2016=pd.read_csv('NBA_2016')

In [3]:
nba_2015= nba[nba.season_end == 2015]
nbaHistoric= nba[nba.season_end <= 2014]

In [4]:
nbaHistoric2= nba[nba.season_end <= 2015]

In [5]:
d = nbaHistoric
d.info()

d_2015 = nba_2015
d_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10965 entries, 0 to 10964
Data columns (total 53 columns):
Unnamed: 0      10965 non-null int64
player          10965 non-null object
pos             10965 non-null object
age             10965 non-null int64
bref_team_id    10965 non-null object
g               10965 non-null int64
gs              10965 non-null int64
mp              10965 non-null float64
fg              10965 non-null float64
fga             10965 non-null float64
fg_             10965 non-null float64
x3p             10965 non-null float64
x3pa            10965 non-null float64
x3p_            10965 non-null float64
x2p             10965 non-null float64
x2pa            10965 non-null float64
x2p_            10965 non-null float64
efg             10965 non-null float64
ft              10965 non-null float64
fta             10965 non-null float64
ft_             10965 non-null float64
orb             10965 non-null float64
drb             10965 non-null float64
trb  

In [55]:
col_u = ["VORP","ORB%","DRB%","FTr","BLK%","TS%","STL%","AST%","3PAr","USG%",'DBPM','OBPM','BPM','PER','DWS','OWS','WS','WS/48','TOV%','TRB%']

In [56]:
features= d[col_u]
features_2015 = d_2015[col_u]
#d.team.replace('None',0,inplace=True)
#labels=pd.to_numeric(d.team,errors='coerce')

In [57]:
#d.team= np.where(d.team != 'None',1,0)
labels = d.team

In [59]:
labels.unique()

array(['None', '2.0', '3.0', '1.0'], dtype=object)

In [60]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3, random_state=1)
train = pd.DataFrame(data=features_train, columns=[col_u])
train['team'] = labels_train
test = pd.DataFrame(data=features_test, columns=[col_u])
test['team'] = labels_test


# Random Forest

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

clf = RandomForestClassifier(n_estimators=100,max_features='auto',min_samples_split=1)

clf = clf.fit(features_train,labels_train)

pred = clf.predict(features_test)

print "Accuracy =", accuracy_score(pred, labels_test)*100,"%"
scores = cross_val_score(clf, features_train, labels_train)
print scores.mean()
clf.feature_importances_

Accuracy = 98.2674772036 %
0.978762347945


array([ 0.0703998 ,  0.03255479,  0.03220734,  0.03482159,  0.0336889 ,
        0.03570255,  0.03680884,  0.0416195 ,  0.0316133 ,  0.07740004,
        0.02768862,  0.03889684,  0.05049652,  0.10668697,  0.05687933,
        0.06531232,  0.10893988,  0.05328851,  0.03318199,  0.03181237])

In [62]:
#features_2015 = d_2015[col_u]
d_2015['pred2015']=clf.predict(features_2015)
d_2015['rfprd']=clf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [63]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd
11202,LeBron James,1.0,1.0,0.25
11077,Stephen Curry,1.0,1.0,0.28
11082,Anthony Davis,1.0,None,0.17
11201,LeBron James,1.0,1.0,0.25
11131,Marc Gasol,1.0,None,0.19
11163,James Harden,1.0,1.0,0.13
10973,LaMarcus Aldridge,2.0,None,0.13
11328,Chris Paul,2.0,1.0,0.18
11437,Russell Westbrook,2.0,1.0,0.17
11132,Pau Gasol,2.0,None,0.27


# Adaboost

In [64]:

from sklearn.ensemble import AdaBoostClassifier
adaclf = AdaBoostClassifier(n_estimators=10)

adaclf = adaclf.fit(features_train,labels_train)

adapred = adaclf.predict(features_test)

print "Accuracy =", accuracy_score(adapred, labels_test)*100,"%"
adascores = cross_val_score(adaclf, features_train, labels_train)
print adascores.mean()
adaclf.feature_importances_

Accuracy = 97.4468085106 %
0.967947674613


array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0.1,  0.2,  0.1,
        0. ,  0. ,  0.2,  0. ,  0. ,  0.3,  0. ,  0. ,  0. ])

In [65]:
#features_2015 = d_2015[col_u]
d_2015['adapredict']=adaclf.predict(features_2015)
d_2015['adaprb']=adaclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [66]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb
11202,LeBron James,1.0,1.0,0.25,None,0.234924
11077,Stephen Curry,1.0,1.0,0.28,2.0,0.260138
11082,Anthony Davis,1.0,None,0.17,2.0,0.260138
11201,LeBron James,1.0,1.0,0.25,None,0.234924
11131,Marc Gasol,1.0,None,0.19,None,0.248511
11163,James Harden,1.0,1.0,0.13,2.0,0.260138
10973,LaMarcus Aldridge,2.0,None,0.13,None,0.234757
11328,Chris Paul,2.0,1.0,0.18,1.0,0.278751
11437,Russell Westbrook,2.0,1.0,0.17,1.0,0.282146
11132,Pau Gasol,2.0,None,0.27,None,0.222883


# KNN

In [67]:
from sklearn.neighbors import KNeighborsClassifier
knnclf = KNeighborsClassifier(n_neighbors=20)

knnclf = knnclf.fit(features_train,labels_train)

knnpred = knnclf.predict(features_test)

print "Accuracy =", accuracy_score(knnpred, labels_test)*100,"%"
knnscores = cross_val_score(knnclf, features_train, labels_train)
print knnscores.mean()


Accuracy = 97.5987841945 %
0.972378219987


In [68]:
#features_2015 = d_2015[col_u]
d_2015['knnpred']=knnclf.predict(features_2015)
d_2015['knnprb']=knnclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb','knnpred','knnprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [69]:
df[df.knnpred!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb,knnpred,knnprb
11077,Stephen Curry,1.0,1.0,0.28,2.0,0.260138,1.0,0.35
11082,Anthony Davis,1.0,None,0.17,2.0,0.260138,1.0,0.15
11163,James Harden,1.0,1.0,0.13,2.0,0.260138,1.0,0.30
11201,LeBron James,1.0,1.0,0.25,None,0.234924,1.0,0.35
11202,LeBron James,1.0,1.0,0.25,None,0.234924,1.0,0.35
11328,Chris Paul,2.0,1.0,0.18,1.0,0.278751,2.0,0.40
11437,Russell Westbrook,2.0,1.0,0.17,1.0,0.282146,1.0,0.35
11012,Sim Bhullar,None,None,0.02,None,0.010984,1.0,0.05


# Decision tree

In [70]:
from sklearn.tree import DecisionTreeClassifier

dtclf = DecisionTreeClassifier(min_samples_split=1)

dtclf = dtclf.fit(features_train, labels_train)

dtpred = dtclf.predict(features_test)

print "accuracy =", dtclf.score(features_test, labels_test) * 100, '%'
dtscores = cross_val_score(dtclf, features_train, labels_train)
print dtscores.mean()
pd.DataFrame({'feature':col_u, 'importance':dtclf.feature_importances_}).sort_values(by='importance',ascending=False)

accuracy = 97.3252279635 %
0.965732860406


,feature,importance
16,WS,0.256664
13,PER,0.101624
1,ORB%,0.057901
17,WS/48,0.051695
14,DWS,0.047693
7,AST%,0.043270
5,TS%,0.042771
18,TOV%,0.041974
3,FTr,0.038170
9,USG%,0.037648


In [71]:
#features_2015 = d_2015[col_u]
d_2015['dtpred']=dtclf.predict(features_2015)
d_2015['dtprb']=dtclf.predict_proba(features_2015)[:, 1]
#pred = clf.predict(features_test)
df=d_2015[['player','team','pred2015','rfprd','adapredict','adaprb','knnpred','knnprb','dtpred','dtprb']]

C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahesh\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [74]:
df[df.team!='None'].sort_values(by='team')

,player,team,pred2015,rfprd,adapredict,adaprb,knnpred,knnprb,dtpred,dtprb
11202,LeBron James,1.0,1.0,0.25,None,0.234924,1.0,0.35,None,0
11077,Stephen Curry,1.0,1.0,0.28,2.0,0.260138,1.0,0.35,1.0,0
11082,Anthony Davis,1.0,None,0.17,2.0,0.260138,1.0,0.15,None,0
11201,LeBron James,1.0,1.0,0.25,None,0.234924,1.0,0.35,None,0
11131,Marc Gasol,1.0,None,0.19,None,0.248511,None,0.20,1.0,0
11163,James Harden,1.0,1.0,0.13,2.0,0.260138,1.0,0.30,1.0,0
10973,LaMarcus Aldridge,2.0,None,0.13,None,0.234757,None,0.10,3.0,0
11328,Chris Paul,2.0,1.0,0.18,1.0,0.278751,2.0,0.40,None,0
11437,Russell Westbrook,2.0,1.0,0.17,1.0,0.282146,1.0,0.35,None,0
11132,Pau Gasol,2.0,None,0.27,None,0.222883,None,0.15,2.0,1


# Naive Bayes

In [27]:
from sklearn.naive_bayes import GaussianNB
nbclf = GaussianNB()

nbclf=nbclf.fit(features_train,labels_train)

nbpred=nbclf.predict(features_test)

print "accuracy =", nbclf.score(features_test, labels_test) * 100, '%'
nbscores = cross_val_score(nbclf, features_train, labels_train)
print nbscores.mean()


accuracy = 90.6990881459 %
0.899934737942


# SVC

In [28]:
from sklearn.svm import SVC
svcclf = SVC(kernel='rbf')

svcclf = svcclf.fit(features_train, labels_train)

svcpred = svcclf.predict(features_test)

print "accuracy =", svcclf.score(features_test, labels_test) * 100, '%'
svcscores = cross_val_score(svcclf, features_train, labels_train)
print svcscores.mean()


accuracy = 97.7811550152 %
0.97355034916
